In [2]:
# !pip install transformers
# !pip install transformers[torch]
# !pip install datasets
!pip install seqeval
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=7facd13d27ede662d369a5fa157c22911ec41cabd77343143050f1309968d742
  Stored in directory: /tmp/pip-ephem-wheel-cache-4dmzoaft/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval

[notice] A new release of pip available: 22.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 5.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 6.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import evaluate

metric = evaluate.load("seqeval")

In [4]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
def get_tag(type, begin):
  if type == "DIAGNOSTICO":
    if begin:
      return 1
    else:
      return 2
  elif type == "PROCEDIMIENTO":
    if begin:
      return 3
    else:
      return 4
  else:
    return 0


def get_token_tags(offset_section, tokens, df_x_train_sel):
  tags = [0 for token in tokens['input_ids']]
  for idx, row in df_x_train_sel.iterrows():
    tuples = get_entity_tuples(row["OFFSETS"])
    tuples_currected = [(item[0] - offset_section, item[1] - offset_section) for item in tuples]

    for entity_span in tuples_currected:
      entity = False
      for idx_offset_mapping, offset_mapping in enumerate(tokens['offset_mapping']):
        if offset_mapping[0] == entity_span[0]:
          entity = True
          # print('match', row["CODE"], row["SUBSTRING"], offset_mapping, entity_span, tokenizer.decode(tokens['input_ids'][idx_offset_mapping]))
          tags[idx_offset_mapping] = get_tag(row["TYPE"], True)
        elif entity:
          if offset_mapping[1] <= entity_span[1]:
            tags[idx_offset_mapping] = get_tag(row["TYPE"], False)
          else:
            entity = False
  return tags
